In [ ]:
#!pip3 install --upgrade gdown
import gdown
url = "https://drive.google.com/uc?id=1_WM9MdokkOR9TsAqy9qHkys24pqeTwMY"
gdown.download(url, quiet=True, use_cookies=False, verify=False)

! unzip -q project.zip

! mv /content/content/folder/project/fake /content/
! mv /content/content/folder/project/real /content/
! mv /content/content/folder/project/features.csv features.csv
! mv /content/content/folder/project/labels.csv labels.csv

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'drive.google.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'drive.google.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(

 	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses. 

You may still be able to access the file from the browser:

	 https://drive.google.com/uc?id=1_WM9MdokkOR9TsAqy9qHkys24pqeTwMY 



Access denied with the following error:
replace content/folder/project/fake/810100457_fake_stable_mountain_2.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: mv: cannot move '/content/content/folder/project/fake' to '/content/fake': Directory not empty
mv: cannot move '/content/content/folder/project/real' to '/content/real': Directory not empty
mv: cannot stat '/content/content/folder/project/features.csv': No such file or directory
mv: cannot stat '/content/content/folder/project/labels.csv': No such file or directory


In [ ]:
from os import walk
import imageio as iio
from tqdm import tqdm
import pandas as pd
import numpy as np
from scipy import stats
import cv2
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
mypath = '/content/content/folder/project/'

In [ ]:
FakeImages = next(walk(mypath + 'fake'), (None, None, []))[2]
FakeImages.remove('desktop.ini')

In [ ]:
RealImages = next(walk(mypath + 'real'), (None, None, []))[2]
RealImages.remove('desktop.ini')

In [ ]:
imgs = list()
for addr in tqdm(FakeImages):
  imgs.append(cv2.imread(mypath + 'fake/' + addr))

100%|██████████| 1710/1710 [00:35<00:00, 48.36it/s]


In [ ]:
for i in tqdm(range(len(imgs))):
  # 1 dimensional pics to 3
  if len(imgs[i].shape) == 2:
    temp = np.array([imgs[i],imgs[i],imgs[i]]).reshape((imgs[i].shape[0], imgs[i].shape[0], 3))
    imgs[i] = temp
  # (3, x, x) to (x, x, 3)
  elif imgs[i].shape[0] < 5:
    imgs[i] = imgs[i].reshape((imgs[i].shape[1], imgs[i].shape[2], imgs[i].shape[0]))

100%|██████████| 1710/1710 [00:00<00:00, 813181.39it/s]


In [ ]:
mean = list()
std = list()
skew = list()
for img in tqdm(imgs):
  mean.append([np.mean(img[:,:,i]) for i in range(3)])
  std.append([np.std(img[:,:,i]) for i in range(3)])
  skew.append([stats.skew(img[:,:,i].reshape((img.shape[0]*img.shape[1]))) for i in range(3)])

100%|██████████| 1710/1710 [01:26<00:00, 19.73it/s]


In [ ]:
mean = list()
std = list()
skew = list()
min = list()
max = list()
names = list()
width = list()
height = list()
for addr in tqdm(FakeImages):
  #Read
  img = cv2.imread(mypath + 'fake/' + addr)
  #
  if not isinstance(img, np.ndarray):
    continue
  names.append(addr)
  #Normalize
  if len(img.shape) == 2:
    img = np.array([img,img,img]).reshape((img.shape[0], img.shape[0], 3))
  elif img.shape[0] < 5:
    img = img.reshape((img.shape[1], img.shape[2], img.shape[0]))
  #Extract Features
  mean.append([np.mean(img[:,:,i]) for i in range(3)])
  min.append([np.min(img[:,:,i]) for i in range(3)])
  max.append([np.max(img[:,:,i]) for i in range(3)])
  std.append([np.std(img[:,:,i]) for i in range(3)])
  skew.append([stats.skew(img[:,:,i].reshape((img.shape[0]*img.shape[1]))) for i in range(3)])
  width.append(img.shape[0])
  height.append(img.shape[1])
  del img
label = ['fake'] * len(mean)

100%|██████████| 1710/1710 [02:00<00:00, 14.23it/s]


In [ ]:
for addr in tqdm(RealImages):
  #Read
  img = cv2.imread(mypath + 'real/' + addr)
  #
  if not isinstance(img, np.ndarray):
    continue
  names.append(addr)
  #Normalize
  if len(img.shape) == 2:
    img = np.array([img,img,img]).reshape((img.shape[0], img.shape[0], 3))
  elif img.shape[0] < 5:
    img = img.reshape((img.shape[1], img.shape[2], img.shape[0]))
  #Extract Features
  mean.append([np.mean(img[:,:,i]) for i in range(3)])
  min.append([np.min(img[:,:,i]) for i in range(3)])
  max.append([np.max(img[:,:,i]) for i in range(3)])
  std.append([np.std(img[:,:,i]) for i in range(3)])
  skew.append([stats.skew(img[:,:,i].reshape((img.shape[0]*img.shape[1]))) for i in range(3)])
  width.append(img.shape[0])
  height.append(img.shape[1])
  del img
label = label + (['real'] * ( len(mean) - len(label) ) )

100%|██████████| 1710/1710 [17:17<00:00,  1.65it/s]


In [ ]:
features = pd.DataFrame()
mean1 = np.array(mean)
min1 = np.array(min)
max1 = np.array(max)
std1 = np.array(std)
skew1 = np.array(skew)
for i in range(3):
  features['mean' + str(i)] = mean1[:,i]
  features['min' + str(i)] = min1[:,i]
  features['max' + str(i)] = max1[:,i]
  features['std' + str(i)] = std1[:,i]
  features['skew' + str(i)] = skew1[:,i]
features['label'] = label
features['file_name'] = names

In [ ]:
features

,mean0,min0,max0,std0,skew0,mean1,min1,max1,std1,skew1,mean2,min2,max2,std2,skew2,label,file_name
0,86.777800,0,255,49.633345,1.076809,99.480340,0,255,53.368603,0.646971,80.690243,0,255,48.343427,1.092130,fake,810100377_fake_stable_jungle_3.jpg
1,202.577752,19,255,47.392084,-1.595573,156.982515,0,255,48.027968,-0.643141,82.878365,0,255,57.074266,0.827175,fake,810698270_fake_dalle_sea_4.jpeg
2,140.681107,0,255,76.235533,0.040739,106.972821,9,255,56.988960,0.844516,84.219409,0,255,62.251163,0.873528,fake,810101168_fake_dalle_mountain_4.jpeg
3,159.373792,6,255,48.349334,-0.511037,130.141318,0,255,42.667631,-0.305628,70.989682,0,255,50.555748,0.959845,fake,810101251_fake_dalle_sea_3.jpeg
4,195.327153,8,255,58.596279,-1.507204,174.101384,2,255,57.920224,-1.179600,142.926207,0,255,64.921639,-0.567142,fake,810199424_fake_dalle_sea_1.jpeg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3412,89.307688,0,255,50.160967,0.424737,102.582437,0,255,56.143832,0.382059,102.108937,0,255,64.471643,0.351969,real,810101293_real_none_jungle_6.jpeg
3413,173.473782,0,207,29.503077,-2.875409,164.541732,0,244,32.386832,-1.755864,128.677943,0,255,57.640221,-0.480218,real,810100284_real_none_sea_7.jpeg
3414,118.071538,0,255,60.794017,-0.374155,109.236126,0,255,52.726437,-0.310408,76.259375,0,255,34.113832,0.833187,real,810101293_real_none_mountain_10.jpeg
3415,130.326711,0,255,40.252426,-0.049243,130.326711,0,255,40.252426,-0.049243,130.326711,0,255,40.252426,-0.049243,real,810100457_real_none_sea_5.jpg


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features[[x for x in features.columns if not x in ['label', 'file_name']]], features['label'], train_size=0.7)

In [ ]:
svmmodel = SVC(C=10000).fit(X_train, y_train)
pred = svmmodel.predict(X_test)
cr = classification_report(y_test, pred).split("\n")
for c in cr:
  print(c)

              precision    recall  f1-score   support

        fake       0.61      0.66      0.63       499
        real       0.65      0.61      0.63       527

    accuracy                           0.63      1026
   macro avg       0.63      0.63      0.63      1026
weighted avg       0.63      0.63      0.63      1026



In [ ]:
nbmodel = GaussianNB().fit(X_train, y_train)
pred = nbmodel.predict(X_test)
cr = classification_report(y_test, pred).split("\n")
for c in cr:
  print(c)

              precision    recall  f1-score   support

        fake       0.55      0.57      0.56       499
        real       0.58      0.56      0.57       527

    accuracy                           0.56      1026
   macro avg       0.56      0.56      0.56      1026
weighted avg       0.56      0.56      0.56      1026



In [ ]:
lrmodel = LogisticRegression(max_iter=1000).fit(X_train, y_train)
pred = lrmodel.predict(X_test)
cr = classification_report(y_test, pred).split("\n")
for c in cr:
  print(c)

              precision    recall  f1-score   support

        fake       0.54      0.59      0.56       499
        real       0.57      0.52      0.55       527

    accuracy                           0.55      1026
   macro avg       0.55      0.55      0.55      1026
weighted avg       0.55      0.55      0.55      1026



In [ ]:
mlpmodel = MLPClassifier(hidden_layer_sizes=[64,16,16,16,16,8]).fit(X_train, y_train)
pred = mlpmodel.predict(X_test)
cr = classification_report(y_test, pred).split("\n")
for c in cr:
  print(c)

              precision    recall  f1-score   support

        fake       0.56      0.77      0.65       499
        real       0.66      0.42      0.51       527

    accuracy                           0.59      1026
   macro avg       0.61      0.59      0.58      1026
weighted avg       0.61      0.59      0.58      1026

